<a href="https://colab.research.google.com/github/boneeyah/DS7337/blob/main/DeepVAR2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [2]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-docker2' for glob 'nvidia*'
Note, selecting 'nvidia-headless-525-server' for glob 'nvidia*'
Note, selecting 'nvidia-headless-530-open' for glob 'nvidia*'
Note, selecting 'nvidia-fs-prebuilt' for glob 'nvidia*'
Note, selecting 'nvidia-driver-530-open' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-headless-no-dkms-515-open' for glob 'nvidia*'
Note, 

In [3]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!sudo add-apt-repository "deb https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/ /"
!sudo apt-get update
!sudo apt-get install cuda-10-2

--2023-06-14 13:00:09--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu1804.pin’

cuda-ubuntu1804.pin 100%[===================>]     190  --.-KB/s    in 0s      

2023-06-14 13:00:09 (6.86 MB/s) - ‘cuda-ubuntu1804.pin’ saved [190/190]

Executing: /tmp/apt-key-gpghome.k0lK31KTrw/gpg.1.sh --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
gpg: requesting key from 'https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub'
gpg: key F60F4B3D7FA2AF80: public key "cudatools <cudatools@nvidia.com>" imported
gpg: Total number processed: 1
gpg:               import

In [4]:
!pip install mxnet-cu102==1.9.1
!pip install gluonts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.8/380.8 MB 3.3 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.8 MB/s eta 0:00:00


In [5]:
import numpy as np
from gluonts.mx.trainer import Trainer
from gluonts.mx.model.deepvar import DeepVAREstimator
from gluonts.dataset.common import ListDataset
import mxnet as mx

/usr/local/lib/python3.10/dist-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [6]:
mx.random.seed(610)
np.random.seed(610)
df_50 = np.loadtxt("https://raw.githubusercontent.com/boneeyah/DS7337/main/nonlinear_synthetic_50.csv",delimiter=",",skiprows=1,usecols=(0,4,5))
df_500 = np.loadtxt("https://raw.githubusercontent.com/boneeyah/DS7337/main/nonlinear_synthetic_500.csv",delimiter=",",skiprows=1,usecols=(0,4,5))
df_2000 =  np.loadtxt("https://raw.githubusercontent.com/boneeyah/DS7337/main/nonlinear_synthetic_2000.csv",delimiter=",",skiprows=1,usecols=(0,4,5))
#h_list = [[3,6],[8,16],[18,36]]
#freq_list = ["M","W","D"]

In [9]:
def model_fit(data_list,horizon_list,freq_list):
  a_preds = {}
  for data in data_list:
    labels = list(set(data[:,0]))
    freq = freq_list[data_list.index(data)]
    for series in labels:
      df = data[data[:,0]==series]
      df = df[:,1:3]
      for h in horizon_list[data_list.index(data)]:
        train = df[:-h]
        test = df[-h:]
        train = ListDataset([{"start":"2000-01-01","target":train.T}],one_dim_target=False,freq=freq)
        estimator = DeepVAREstimator(freq=freq, prediction_length=h,target_dim=2,scaling=True,num_layers=4, trainer = Trainer(ctx = 'gpu'))#trainer = Trainer(ctx='gpu')
        predictor = estimator.train(training_data=train)
        preds = predictor.predict(train,num_workers = 4)
        preds = list(preds)[0].quantile(0.5)
        a_preds.update({"Data{}{}_h{}".format(data_list.index(data),series,h):{"preds":preds.tolist(),"test":test.tolist()}})
  return(a_preds)

In [10]:
results_50 = model_fit([df_50],[[3,6]],["M"])
results = open('results_50.txt','w')
results.write(str(results_50))
results.close()

100%|██████████| 50/50 [00:02<00:00, 18.77it/s, epoch=100/100, avg_epoch_loss=-1.11]


In [ ]:
results_500 = model_fit([df_500],[[8,16]],["W"])
results = open('results_500.txt','w')
results.write(str(results_500))
results.close()

In [ ]:
results_2000 = model_fit([df_2000],[[18,36]],["D"])
results = open('results_2000.txt','w')
results.write(str(results_2000))
results.close()